In [1]:
import numpy as np
import time
import sys
import os
from joblib import Parallel, delayed
import multiprocessing

Copied AddFP script here for debug purpose.

In [11]:
# f.write('Adding phonon configuration.\n\n\n')
base_name = 'Al2O3'
folder = '/srv/home/chenyu/Al2O3_simulation/Test/'
cbed_list_unique = [img for img in os.listdir(folder) if (base_name in img) and ('_FPavg.npy' in img) and ('_X1_Y1_' in img)]
nlayer = len(cbed_list_unique)
num_cores = multiprocessing.cpu_count()
# f = open("output.txt","a")
# print(num_cores)
print("Total " + str(num_cores) + " cores in use for " + str(nlayer) + " layers.\n")
Parallel(n_jobs=min(num_cores,nlayer))(delayed(AddFP)(folder,base_name,ilayer) for ilayer in range(nlayer))

Total 40 cores in use for 126 layers.



[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [10]:
def AddFP(folder, base_name, ilayer):
    layer_name = '_slice'+ str(ilayer) + '_'
#     f = open("output.txt","a")
    print("Now working on layer " + str(ilayer) + "\n")
    # only work for CBEDs in slice ilayer
    cbed_list_unique = [img for img in os.listdir(folder) if (base_name in img) and ('FP1.mrc' in img) 
                        and (layer_name in img)]    # find all mrc files for ilayer
    # shape = readCBEDfromMRC(folder + cbed_list_unique[0]).shape
    
    for pos in cbed_list_unique:
        # create one empty npy file for each mrc file with FP1
        cmp = pos.rsplit('_',1)
        cbed_pos = cmp[0][:]
        tmp_array = readCBEDfromMRC(folder + pos)
        np.save(folder + cbed_pos + '_FPavg',tmp_array + np.load(folder+cbed_pos+'_FPavg.npy'))  # replace old FPavg.npy file with new one
        os.remove(folder + pos)


    numfiles = len(cbed_list_unique)
    print("Layer " + str(ilayer) + " finished, totally " + str(numfiles) + "files processed.\n" )
#     f.close()

In [3]:
def readCBEDfromMRC(fname):
    f = open(fname, 'rb')
    count = 10
    b = f.read(count*4)
    data_types = {0:np.uint8,1:np.intc,2:np.float32,6:np.uint16}
    c = np.frombuffer(b, dtype=np.int32,count=count)
    data_size = c[:3]
    data_type = data_types[c[3]]

    f.seek(1024)

    count = data_size[0]*data_size[1]*data_size[2]
    cbed = f.read(count*4)
    cbed_arr = np.frombuffer(cbed,dtype=data_type,count=count)
    cbed_arr = np.reshape(cbed_arr,tuple(data_size))
    f.close
    f.closed
    return cbed_arr